In [6]:
import pandas as pd
import json
import pickle

In [7]:
with open(r"C:\Users\deviy\Desktop\reco\Foodfinder-Recommendation-System\reco_data\yelp_academic_dataset_business.json", encoding="utf8") as business_file:
    business_data = [json.loads(line) for line in business_file]
business_df = pd.DataFrame(business_data)

In [2]:
# Load user data
with open(r"C:\Users\deviy\Desktop\reco\Foodfinder-Recommendation-System\reco_data\yelp_academic_dataset_user.json", encoding="utf8") as user_file:
    user_data = [json.loads(line) for line in user_file]
user_df = pd.DataFrame(user_data)

In [3]:
# Load review data
with open(r"C:\Users\deviy\Desktop\reco\Foodfinder-Recommendation-System\reco_data\yelp_academic_dataset_review.json", encoding="utf8") as review_file:
    review_data = [json.loads(line) for line in review_file]
review_df = pd.DataFrame(review_data)

**Filter Loc&Categ of Business**

In [18]:
nash_business_df=business_df.loc[business_df['city']=='Nashville']

In [21]:
nash_business_df.categories.value_counts()

Restaurants, Mexican                                                                                       44
Mexican, Restaurants                                                                                       35
Beauty & Spas, Nail Salons                                                                                 35
Nail Salons, Beauty & Spas                                                                                 28
Hotels & Travel, Event Planning & Services, Hotels                                                         28
                                                                                                           ..
Restaurants, Food, Beverage Store, Pizza, Italian                                                           1
Real Estate, Banks & Credit Unions, Local Services, Home Services, Financial Services, Mortgage Brokers     1
Garage Door Services, Local Services, Contractors, Home Services                                            1
Shopping, 

In [22]:
len(nash_business_df)#.categories.value_counts()

6971

In [23]:
del business_df

In [28]:
# Filter Nashville restaurants and food-related businesses
nashville_restaurants_df = nashville_business_df[
    nashville_business_df['categories'].str.contains('Restaurants|Food') & 
    nashville_business_df['categories'].notna()
]

In [30]:
with open('nash_restaurants.pkl','wb') as f:          
    pickle.dump(nashville_restaurants_df,f)

In [34]:
# Get list of Nashville restaurant business IDs
nashville_restaurant_ids = nashville_restaurants_df['business_id'].tolist()

In [35]:
nashville_restaurant_ids 

['bBDDEgkFA1Otx9Lfe7BZUQ',
 '4iRzR7OaS-QaSXuvYxEGKA',
 'tMkwHmWFUEXrC9ZduonpTg',
 'lk9IwjZXqUMqqOhM774DtQ',
 'oaboaRBUgGjbo2kfUIKDLQ',
 'X_E7U2lVNEwbH_gpOBmAHQ',
 'G4lRmwDHdvhZAhmtlsYYBw',
 'ORL4JE6tz3rJxVqkdKfegA',
 '1dSKEitDDgIkaApe6UNMSA',
 '42OM20QndpmsOSY6ccudBA',
 'H3JjbID9Zhukpqr9uupHAA',
 'dD2p903p8lU0IgXT3OFluA',
 '6mCsrUp9Vl4iXMbOxWtVjA',
 'Gki1ureeRKqlIkQ-AELDVg',
 '5EGhvgDt_E-JPl4EhQkOtg',
 'c6xOzTbNqD-1g9PoxAh4DQ',
 'I2CxrvM0BGH1svfxqGGD2g',
 'g7GCbnZC_HKO0ybNt7NXBw',
 'ByTZJsTDDhvOYKCj7uoy4w',
 'RJPRi1pwocHNZr9ISz_P-A',
 'E3xBC7Bb6KRlZeqcY3bWtA',
 '4Rz9e2RmP7ErILpcls-n2w',
 'RzGr37s8p2ujVuN9of7ksQ',
 '8SqvtaRu9likNDAfAU-HBQ',
 '7CBsFEzEny_Ro833RS6Acg',
 'diqXZ9CBFlIHW7i9UHefhw',
 'sway6xsRT_CUC6peK_Pyww',
 '_49FsmTfruZSTjxU4OwOsg',
 'J7Oo9rhwX7O3ak54pu_Mhg',
 '8s1uT8dyBEUwTNeHoZntMg',
 '_cndcbuaqSRW5f-MeZasNw',
 'zxpJ4Frr8Zoru-YaNzOxzw',
 'qZIiURCjON7QcTzNtG9JpQ',
 's3FRhtaNU8dt-mKkLKUQaQ',
 'KPmFv_Ou6ddauV_gYauWzg',
 'ZJC0ONdxmIlwidXd8E0fXg',
 'k7oZJB7BSx37e-masOmiLQ',
 

In [36]:
with open('nash_resto_ids.json', 'w') as fp:
    json.dump(nashville_restaurant_ids, fp)

In [37]:
# Get list of open Nashville restaurant business IDs
nashville_open_restaurant_ids = nashville_restaurants_df[nashville_restaurants_df['is_open'] != 0]['business_id'].tolist()

In [38]:
with open('nash_open_ids.json', 'w') as fp:
    json.dump(nashville_open_restaurant_ids, fp)

**Filter important users & Reviews**

In [25]:
important_users_df = user_df[(user_df['review_count'] >= 10) | (user_df['elite'].apply(lambda x: len(str(x)) > 0))]

In [40]:
# Get list of important user IDs
important_user_ids = important_users_df['user_id'].tolist()

In [26]:
del user_df

In [41]:
# Filter reviews for important users and Nashville restaurants
nashville_reviews_df = review_df[
    review_df['user_id'].isin(important_user_ids) &
    review_df['business_id'].isin(nashville_restaurant_ids)
]


In [42]:
# Calculate average stars for each Nashville restaurant
nashville_stars_df = nashville_reviews_df[nashville_reviews_df['business_id'].isin(nashville_restaurants_df['business_id'])].groupby('business_id')['stars'].mean().reset_index()

In [43]:
with open('nash_stars.pkl','wb') as f:          
    pickle.dump(nashville_stars_df, f)

In [49]:
# Filter important users who have reviewed Nashville restaurants
nashville_important_users_df = important_users_df[important_users_df['user_id'].isin(nashville_reviews_df['user_id'])]

In [45]:
with open('nash_reviews.pkl','wb') as f:          
    pickle.dump(nashville_reviews_df, f)

In [50]:
with open('nash_user.pkl','wb') as f:         
    pickle.dump(nashville_important_users_df, f)

In [52]:
nashville_important_users_df.shape

(73311, 22)